In [ ]:
import gzip
from io import BytesIO
import numpy as np
import pandas as pd
import os
import requests
import json
import csv
import shutil
from datetime import date, timedelta

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

In [ ]:
from preprocessing_utils import preprocess_NVD_data

In [ ]:
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)

# EPSS data

Download the EPSS data from https://www.first.org/epss/data_stats into `data` folder

In [ ]:
base_url = "https://epss.empiricalsecurity.com/epss_scores-"
date_current = str(date.today())
epss_url = base_url + date_current + ".csv.gz"
epss_filename = "epss_scores-today.csv"

response = requests.get(epss_url)
with open(os.path.join(data_path, epss_filename), "wb") as f:
    f.write(gzip.decompress(response.content))

In [ ]:
epss_current = pd.read_csv(os.path.join(data_path, epss_filename), header=1)
epss_current  # a Python statement with a variable name at the end of a cell will display its contents below


# NVD data

In [ ]:
base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"
date_start_NVD = '2025-09-01T00:00:00.000Z'  # Do NOT change these dates
date_end_NVD = '2025-10-01T00:00:00.000Z'  # Do NOT change these dates
start_index = 0
results_per_page = 1000
total_results = 1

candidate_cves = []
while start_index < total_results:
    params = {
        "pubStartDate": date_start_NVD,
        "pubEndDate": date_end_NVD,
        "resultsPerPage": results_per_page,
        "startIndex": start_index,
        "noRejected": ""
    }
    response = requests.get(base_url, params=params, timeout=6)
    if response.status_code != 200:
        print("Error:", response.status_code)
        break
    data = response.json()
    total_results = data.get("totalResults", 0)
    candidate_cves.extend(data.get("vulnerabilities", []))
    start_index += results_per_page
    print(start_index)

In [ ]:
# normalize and preprocess data
candidate_cves_df = pd.json_normalize(candidate_cves, record_path=None, sep='.', max_level=None)
candidate_cves_df = preprocess_NVD_data(candidate_cves_df)

# remove vulnerabilities marked as "reject" or "reserved"
candidate_cves_df = candidate_cves_df[(candidate_cves_df['cve.vulnStatus'] != 'Reserved') & (candidate_cves_df['cve.vulnStatus'] != 'Reject')]

# merge NVD and EPSS data
candidate_cves_df = candidate_cves_df.merge(epss_current, left_on="cve.id", right_on="cve", how="left")

In [ ]:
# save nvd data
with open(os.path.join(data_path, "candidate_cves.json"), "w", encoding="utf-8") as f:
    json.dump(candidate_cves, f, indent=2)

# save the final dataframe
candidate_cves_df.to_csv(os.path.join(data_path, "candidate_cves_df.csv"))

# Exploratory Data Analysis

- display some examples (e.g., the first two CVE records)

In [ ]:
candidate_cves_df.head(2).T

- show a bar plot with the daily volume of published CVEs

In [ ]:
published_counts = candidate_cves_df["cve.published"].dt.date.value_counts().sort_index()

plt.figure(figsize=(12, 5))
sns.barplot(x=published_counts.index, y=published_counts.values, color="k")
plt.xticks(rotation=90)
plt.xlabel("Date")
plt.ylabel("Number of CVEs Published")
plt.title("CVE Publications per Day")
plt.tight_layout()
plt.show()

- print the description of the last ten published vulnerabilities

In [ ]:
for idx, x in enumerate(candidate_cves_df.sort_values('cve.published', ascending=False)[:10].iterrows()):
    print('-' * 100)
    print(x[1]['cve.id'], x[1]['cve.published'])
    print(x[1].description)


### <font color='blue'><b><i>TODO</i></b>: produce plots or tables to address the folowing points</font>
- what is the percentage of CVEs which received a CVSS score?
- report descriptive statistics of CVSS the CVSS base score and/or show its distribution
- report descriptive statistics of EPSS and/or show its distribution
- produce a scatter plot showing CVSS vs EPSS
- <b>be creative</b>!
    - How many vulnerabilities are published on CISA KEV? 
    - What are the the 20 most frequent vendors? (vendor name can be extracted from the `vulnerable_cpes` field).
    - What are the 20 most frequent CWEs?
    - Anaything else you see fit!

<font color='blue'>Use text cells to discuss the outcome after each point</font>

What is the percentage of CVEs which received a CVSS score?

In [ ]:
print(f"{(candidate_cves_df["cvss_baseScore"].count() / len(candidate_cves_df)) * 100:.02f}%")

# CVE selection


### <font color='blue'><b><i>TODO</i></b>
- Filter the CVEs with low EPSS (<1%)
- Select candidate CVEs
    - From the resulting subset, select 10 CVEs that you think will reach high EPSS by the end of the course.
    - Clearly describe the criteria you used for selection (e.g., high CVSS, popular software, CWE, popular vendor, number of references, keyword in description, manual inspection, random sampling, security blogs).
- Share the selected CVE ids with the instructor (by two weeks). Use the code cell below to produce the csv file to submit.
- Track the EPSS of your CVEs over time


As per specification, we start by filtering the CVEs with low EPSS (<1%)

In [ ]:
candidate_cves_df = candidate_cves_df[candidate_cves_df['percentile'] <= 0.01]
candidate_cves_df.info()

## NVD complete database
We start by downloading all the CVEs that have ever been published between 2002 and 2024

In [ ]:
base_url = "https://nvd.nist.gov/feeds/json/cve/2.0/nvdcve-2.0-"
years = range(2002, 2025, 1)
ext = ".json.gz"

all_cves = []

for year in years:
    # if json file exists, load it instead and break cycle
    csv_url = base_url + str(year) + ext
    response = requests.get(csv_url)
    if response.status_code != 200:
        print("Error:", response.status_code)
        break
    with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
        all_cves.extend(json.load(f).get("vulnerabilities", []))
    with open(os.path.join(data_path, "all_cves.json"), "ab") as f:
        f.write(gzip.decompress(response.content))
    with open(os.path.join(data_path, "all_cves.json"), "at") as f:
        f.write("\n")
# is json file doesn't exist use json to store all_cves inside it

### Data Cleaning

In [ ]:
# normalize and preprocess data
all_cves_df = pd.json_normalize(all_cves, record_path=None, sep='.', max_level=None)
all_cves_df = preprocess_NVD_data(all_cves_df)

# remove vulnerabilities marked as "reject" or "reserved"
all_cves_df = all_cves_df[(all_cves_df['cve.vulnStatus'] != 'Reserved') & (all_cves_df['cve.vulnStatus'] != 'Reject')]

all_cves_df.describe()

In [ ]:
all_cves_df.isnull().sum()

Some features have a high number of missing values, so we drop the columns directly. Due to the sheer amount of samples, we also remove all the rows that have a missing value. Since we will aggregate data from all the CVEs' histories, we also drop date-related columns.

In [ ]:
X = all_cves_df.drop(columns=["cve.evaluatorSolution", "cve.evaluatorImpact", "cve.vendorComments", "cve.evaluatorComment", "cve.cisaExploitAdd", "cve.cisaActionDue", "cve.cisaRequiredAction", "cve.cisaVulnerabilityName", "cve.published", "cve.lastModified"]).dropna()
X.info()

In [ ]:
cat_features = []
num_features = ["num_references", "cvss_baseScore"]

### EDA

We start by looking at interesting correlations, if there are any.

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(X[["num_references", "cvss_baseScore"]].corr(), annot=False, cmap="coolwarm")
plt.title("Correlation Matrix of CVE Features")
plt.show()

As we can see, the only two numerical features aren't really correlated with each other. We are done for now, since we do not have integrated the calculated target(s) to the dataset.

## Feature construction from historical EPSS data

For each CVE, we download its complete EPSS history and we determine if at any point it satisfied the metrics set by the exercise. Since the metrics are 2, we will append 2 binary labels accordingly.

In [ ]:
current_date = date(2021, 4, 14)
while current_date <= date.today():
    url = "https://epss.empiricalsecurity.com/epss_scores-{:%Y-%m-%d}.csv.gz".format(current_date)
    filename = os.path.join(data_path, f"epss_scores-{current_date:%Y-%m-%d}.csv.gz")

    # Skip if already downloaded
    if os.path.exists(filename):
        print(f"Skipping {filename} (already exists)")
    else:
        print(f"Downloading {url}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Saved to {filename}")
        else:
            print(f"No file for {current_date:%Y-%m-%d} (HTTP {response.status_code})")

    current_date += timedelta(days=1)

print("Download complete.")

In [ ]:
# Loop through all .gz files
for filename in os.listdir(data_path):
    if filename.endswith(".csv.gz"):
        gz_path = os.path.join(data_path, filename)
        csv_path = os.path.join(data_path, filename[:-3])  # Remove .gz

        print(f"Unzipping {gz_path} -> {csv_path}")
        with gzip.open(gz_path, "rb") as f_in:
            with open(csv_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)

print("All files unzipped successfully.")

For each CVE, we extract its time series of percentiles and we check if it ever satisfied our metrics

In [ ]:
def max_mean_daily_gain(values):
    # Maximum mean daily gain over all 90-day windows
    max_mean_gain = float('-inf')
    for i in range(len(values) - 89):
        window = values[i:i+90]
        window_sum = 0
        for j in range(1, 90):
            window_sum += window[j] - window[0]
        mean_gain = window_sum / 90
        max_mean_gain = max(max_mean_gain, mean_gain)
    return max_mean_gain

def max_total_gain(values):
    # Maximum total gain over all 90-day windows
    max_gain = float('-inf')
    for i in range(len(values) - 89):
        window = values[i:i+90]
        for j in range(1, 90):
            max_gain = max(max_gain, window[j] - window[0])
    return max_gain

# Configuration
cveN = "CVE-2021-34527"  # Replace with the desired CVE number

percentile_values = []
for cve in X:
    cveN = cve[""]
# Loop through all CSV files
for filename in sorted(os.listdir(data_path)):
    if filename.endswith(".csv") and filename.startswith("epss_scores-"):
        file_path = os.path.join(data_path, filename)

        # Extract date from filename
        date_part = filename[len("epss_scores-"):-len(".csv")]

        # Read file and search for CVE
        with open(file_path, newline='', encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if len(row) >= 3 and row[0].strip() == cveN:
                    value = float(row[2])
                    percentile_values.append(value)
                    break  # Stop after first match in this file

In [ ]:
nickname = 'template_submsission'  # TODO: put your nickname here

# TODO: put your selected IDs here
selected = ['CVE-YYYY-XXXXX0',
            'CVE-YYYY-XXXXX1',
            'CVE-YYYY-XXXXX2',
            'CVE-YYYY-XXXXX3',
            'CVE-YYYY-XXXXX4',
            'CVE-YYYY-XXXXX5',
            'CVE-YYYY-XXXXX6',
            'CVE-YYYY-XXXXX7',
            'CVE-YYYY-XXXXX8',
            'CVE-YYYY-XXXXX9',
            ]

candidate_cves_df[candidate_cves_df['cve.id'].isin(selected)].to_csv(os.path.join(data_path, f'{nickname}.csv'))